将train/val/test.npz -> data/index.npz

In [ ]:
'''
train/val/test.npz: x,y [B,T_in,N,D] [B,T_out,N,D] 
data.npz: data [B,N,D]   index.npz: train/val/test [B,3] 
'''

In [ ]:
import numpy as np
data_name = 'NYCTaxi' #'NYCBike1' 'NYCBike2' 'NYCTaxi' 'METRLA' 'PEMSBAY'
data_train = np.load(f'./{data_name}/train.npz')
data_val = np.load(f'./{data_name}/val.npz')
data_test = np.load(f'./{data_name}/test.npz')

In [ ]:
x_train = data_train['x']
y_train = data_train['y']
x_val = data_val['x']
y_val = data_val['y']
x_test = data_test['x']
y_test = data_test['y']

1. 获取data.npz

In [ ]:
def merge_x_y(x_data, y_data):
    merged = np.concatenate((x_data, y_data), axis=1)
    return merged

train_merged = merge_x_y(x_train, y_train)
val_merged = merge_x_y(x_val, y_val)
test_merged = merge_x_y(x_test, y_test)

all_data = np.concatenate((train_merged, val_merged, test_merged), axis=0) # [B, T, N, D]
print(all_data.shape)

In [ ]:
all_data = all_data.reshape(-1, all_data.shape[2], all_data.shape[3])  # (B*T, N, D)
print(all_data.shape)

np.savez_compressed(f'./{data_name}/data.npz', data=all_data)

2. 获取index.npz


In [ ]:
num_train = data_train['x'].shape[0]
num_val = data_val['x'].shape[0]
num_test = data_test['x'].shape[0]

time_interval = 20 if data_name=='NYCBike1' else 36
# 创建索引函数，添加了起始索引
def create_indices(num_samples, time_interval, start_index=0):
    indices = np.zeros((num_samples, 3), dtype=int)
    for i in range(num_samples):
        base_index = start_index + i * time_interval
        indices[i, 0] = base_index 
        indices[i, 1] = base_index + time_interval - 1
        indices[i, 2] = base_index + time_interval
    return indices


start_index_val = num_train * time_interval
start_index_test = start_index_val + num_val * time_interval

train_indices = create_indices(num_train, time_interval)
val_indices = create_indices(num_val, time_interval, start_index_val)
test_indices = create_indices(num_test, time_interval, start_index_test)

np.savez_compressed(f'./{data_name}/index.npz', train=train_indices, val=val_indices, test=test_indices)